In [ ]:
train = activity.query('date <= "2025-01-20"')
test = activity.query('date > "2025-01-20"').astype({'DAU' : 'int64', 'view' : 'int64'})


In [ ]:
dlt_map = DLT(response_col="DAU", #название колонки с метрикой
          date_col="date", #название колонки с датами-временем
          seasonality=7, #длина периода сезонности если у нас дневные данные - то очевидно что есть недельный цикл 
          estimator="stan-map", #алгоритм оценки
          n_bootstrap_draws=3000) #количество сэмплов бутстрапа для доверительного интервала

In [ ]:
#Обучим модель на периоде train
dlt_map.fit(train)
#Предсказываем на период test
predicted_df_map = dlt_map.predict(df=test)

In [ ]:
#Рисуем прогноз на 3 недели (обучение модели 1,5 месяца)
_ = plot_predicted_data(activity, predicted_df_map, "date", 'DAU', title='Prediction with DLT')

In [ ]:
# Посмотрим на отдельные компоненты модели
predicted_df_map = dlt_map.predict(df=test, decompose=True) # декомпозиция предсказания
_ = plot_predicted_components(predicted_df_map, "date", plot_components=['prediction', 'trend', 'seasonality'])

In [ ]:
# Строим модель (MCMC) как более точную
dlt_mcmc = DLT(seasonality=7, response_col="DAU", date_col="date", 
               estimator='stan-mcmc', #новый алгоритм оценки
               num_warmup=4000,       #время "разогрева"
               num_sample=4000)       #время сэмплирования

In [ ]:
# Построим модель с регрессором - количество просмотров, предварительно посмотрим корреляцию
dlt_mcmc_reg = DLT(seasonality=7, response_col="DAU", date_col="date", 
               estimator='stan-mcmc', #новый алгоритм оценки
               num_warmup=4000,       #время "разогрева"
               num_sample=4000,       #время сэмплирования
               regressor_col=["view"]) #наша колонка с регрессором

In [ ]:
# Попробуем построить модель с регуляризацией 'auto_ridge'
dlt_mcmc_reg_auto = DLT(seasonality=7, response_col="DAU", date_col="date", 
               estimator='stan-mcmc', #новый алгоритм оценки
               num_warmup=4000,       #время "разогрева"
               num_sample=4000,       #время сэмплирования
               regressor_col=["view"], #наша колонка с регрессором
               regression_penalty='auto_ridge'      )  

In [ ]:
#для подбора оптимальных параметров
from orbit.utils.params_tuning import grid_search_orbit #для подбора оптимальных параметров

best_params, tuned_df = grid_search_orbit(param_grid={"damped_factor": np.arange(0.1, 1.0, 0.1)}, #наш параметр
                                        model=dlt_mcmc,
                                        df=activity,
                                        min_train_len=7*2,
                                        incremental_len=7,
                                        forecast_len=7,
                                        metrics=None, #смотрим smape по умолчанию
                                        criteria="min") #говорим, что нам нужна минимизаци

In [ ]:
# Посмотрим на ошибку rmsse моделей
from orbit.diagnostics.metrics import rmsse
print(f"Модель МАР: {rmsse(test.DAU.values, predicted_df_map.prediction.values, train.DAU.values)}")
print(f"Модель МСМС: {rmsse(test.DAU.values, predicted_df_mcmc.prediction.values, train.DAU.values)}")
print(f"Модель МСМС+регрессор: {rmsse(test.DAU.values, predicted_df_mcmc_reg.prediction.values, train.DAU.values)}")
print(f"Модель МСМС+регрессор auto: {rmsse(test.DAU.values, predicted_df_mcmc_reg_auto.prediction.values, train.DAU.values)}")
print(f"Модель МСМС+регрессор lasso: {rmsse(test.DAU.values, predicted_df_mcmc_reg_lasso.prediction.values, train.DAU.values)}")